# Text Classification Using Transformer Networks (BERT)

Some initialization:

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

In [3]:
def read_dota_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["text","target"]]

In [4]:
labels = open('classes.txt').read().splitlines()
data = read_dota_data('tagged-data.csv')
print(labels)
data = data.rename(columns={"target": "label"})
data

['0', '1', '2']


,text,label
0,COMMEND ME TY,0
1,sorry nex,0
2,what is the best soup?,0
3,man that silence on axe,0
4,not coming into play,0
...,...,...
3262,"wt?f?asfU JGOFIDLK,YH",1
3263,you must really suck,2
3264,YOU HAVE IDIOT PLAYER,2
3265,SUPER IDIOT,2


In [5]:
from sklearn.model_selection import train_test_split

train_df, eval_and_test_df = train_test_split(data, train_size=0.8, random_state=seed)
eval_df, test_df = train_test_split(eval_and_test_df, train_size=0.5, random_state=seed)
train_df.reset_index(inplace=True, drop=True)
eval_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'train rows: {len(train_df.index):,}')
print(f'eval rows: {len(eval_df.index):,}')
print(f'test rows: {len(test_df.index):,}')

train rows: 2,613
eval rows: 327
test rows: 327


In [6]:
eval_df

,text,label
0,youtube.com/mirocj01 -----> Divine crusher,0
1,u done nothing,0
2,end please,0
3,WHO WILL SUCK DICK AGTER,2
4,sry only noobs here,2
...,...,...
322,you are not pro btw,0
323,just report them,0
324,He asked for a pause to recconect,0
325,and suck my dick,2


In [7]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df)
ds['validation'] = Dataset.from_pandas(eval_df)
ds['test'] = Dataset.from_pandas(test_df)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2613
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
})

Tokenize the texts:

In [8]:
from transformers import AutoTokenizer

transformer_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

In [9]:
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/2613 [00:00<?, ? examples/s]

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 1108, 4106, 2286, 24438, 1186, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
1,2,"[101, 17480, 1801, 1518, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
2,2,"[101, 1143, 1324, 117, 1714, 11348, 1111, 2243...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,0,"[101, 124, 1110, 1268, 1397, 1106, 125, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
4,0,"[101, 1192, 1132, 1226, 1104, 1115, 1679, 1358...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
2608,0,"[101, 3869, 136, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2609,2,"[101, 1228, 28015, 1116, 10696, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
2610,0,"[101, 1195, 1136, 1138, 126, 6485, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
2611,0,"[101, 1198, 2999, 1342, 1155, 1444, 3858, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"


Create the transformer model:

In [10]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [11]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    num_labels=len(labels),
    # num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create the trainer object and train:

In [12]:
from transformers import TrainingArguments

num_epochs = 10
batch_size = 24
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    weight_decay=weight_decay,
)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

/tmp/ipykernel_2748/485909320.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.420897,0.825688
2,No log,0.461306,0.837920
3,No log,0.642480,0.834862
4,No log,0.925596,0.828746
5,0.253300,0.779223,0.847095
6,0.253300,1.069924,0.840979
7,0.253300,1.054954,0.853211
8,0.253300,1.066494,0.853211
9,0.253300,1.075270,0.856269
10,0.015400,1.090788,0.856269


TrainOutput(global_step=1090, training_loss=0.12346708300190234, metrics={'train_runtime': 219.6932, 'train_samples_per_second': 118.939, 'train_steps_per_second': 4.961, 'total_flos': 261023413294602.0, 'train_loss': 0.12346708300190234, 'epoch': 10.0})

Evaluate on the test partition:

In [16]:
test_ds = ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds.to_pandas()

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,2,"[101, 6100, 2592, 1142, 1266, 26144, 3676, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 7642, 5773, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2,0,"[101, 1710, 136, 136, 136, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
3,0,"[101, 189, 1548, 18257, 2956, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
4,2,"[101, 1534, 1183, 2087, 1665, 21569, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
322,0,"[101, 1231, 1643, 171, 1324, 1126, 23609, 8484...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
323,2,"[101, 3919, 14084, 4894, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
324,0,"[101, 126, 1904, 3932, 1297, 13894, 5815, 1162...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
325,0,"[101, 1119, 1116, 1435, 1158, 1171, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"


In [17]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[-3.7721727 , -2.5061538 ,  7.3513947 ],
       [ 7.2128425 , -3.369755  , -4.383509  ],
       [ 7.583261  , -3.9638357 , -4.625444  ],
       [ 7.28931   , -3.6370335 , -4.5734797 ],
       [ 5.3447394 , -1.1218708 , -4.2417474 ],
       [ 7.106233  , -4.321277  , -3.1568577 ],
       [-2.6519184 , -0.8669211 ,  4.4226723 ],
       [-4.098985  , -2.3579907 ,  7.3413773 ],
       [ 6.817859  , -3.2260742 , -4.5264864 ],
       [ 7.346589  , -4.0083294 , -3.9434352 ],
       [-4.6294475 ,  1.708187  ,  3.0579116 ],
       [ 6.8699574 , -4.827702  , -2.7850754 ],
       [-3.5166779 , -2.652137  ,  7.411831  ],
       [ 7.1392107 , -3.0693157 , -4.852673  ],
       [-3.5756934 ,  6.994259  , -2.5180461 ],
       [ 7.600855  , -3.8550491 , -4.533127  ],
       [-3.492923  , -2.8319182 ,  7.3884897 ],
       [-4.591824  ,  6.1136117 , -1.064973  ],
       [ 7.612858  , -4.044943  , -4.5029607 ],
       [ 5.447721  , -5.0534368 , -0.47124857],
       [ 0.

In [18]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic","kinda toxic","toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.94      0.94      0.94       201
 kinda toxic       0.64      0.59      0.61        39
       toxic       0.71      0.75      0.73        87

    accuracy                           0.85       327
   macro avg       0.77      0.76      0.76       327
weighted avg       0.85      0.85      0.85       327

